# Macros 

#### Recap

Last time we spoke about how syntax is represented in Julia. 

Recall, Julia syntax is represented as an `Expr` object.

In [6]:
dump(:(1+2))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 2


Working with `Expr`s directly is a bit yucky, so we can use *quoting* instead.

In [10]:
quote 
    1+2
end
# or 
expr = :(1+2)

:(1 + 2)

or strings

In [13]:
Meta.parse("1+2")

:(1 + 2)

Interpolation can be used to include values into expressions, i.e. 

In [15]:
z=1
expr = :($z+2)

:(1 + 2)

Variales are represented as `Symbol`s

In [17]:
:(x+y)

:(x + y)

Lasty, we evaluate expressions with 

In [19]:
eval(expr)

3

This executes in the *global* scope

In [24]:
c=0
ex = :(c+=1)
function incerement_10()
    for n in 1:10
        eval(ex)
    end
    return nothing
end
incerement_10()
c

10

Wierd! 

### why?
There are advantages to representing syntax internally (as opposed to as a string) -- introsepection, more conveninent destructuring of code. For most use cases, it doesn't matter. 

### Uses
Typically, we can achieve all we need with normal julia code, but metaprogramming can come in handy for 

Automatic code generation -- I've used it for testing or generalising methods for differenet types. 

In [26]:
struct my_angle
    a::Float64
end

for f in (:sin, :cos, :tan, :tanh, :cosh, :sinh, :exp)
    @eval Base.$f(x::my_angle) = my_angle($f(x.a))
end
a = my_angle(1.0)
sin(a)

my_angle(0.8414709848078965)

# Uses: Macros

With metaprogramming we can create `function`s to manipulate `Expr`essions and then `eval`uate them at run-time. 

So what is the point in Macros?